In [0]:
# Import the dependencies
import tensorflow as tf
 
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D,\
                                 concatenate, Dropout, GlobalMaxPooling2D
from tensorflow.keras.applications.imagenet_utils import preprocess_input  
from keras_applications.imagenet_utils import _obtain_input_shape
from tensorflow.keras import backend as K
 
#2.2.0
print(tf.__version__)

2.2.0


In [0]:
class I_Block():

  def __init__(self, filters, kernel_size, stage, block):
    self.filters = filters
    self.kernel_size = kernel_size
    self.conv_name_base = 'res' + str(stage) + block + '_branch'
    self.bn_name_base = 'bn' + str(stage) + block + '_branch'
    self.conv2D_1 = Conv2D(filters[0], kernel_size= (1, 1), strides=(1,1), padding= "valid", name=self.conv_name_base + '2a')
    self.conv2D_2 = Conv2D(filters[1], kernel_size= (kernel_size, kernel_size), strides=(1,1), padding= "same", name=self.conv_name_base + '2b')
    self.conv2D_3 = Conv2D(filters[2], kernel_size= (1, 1), strides=(1,1), padding= "valid", name=self.conv_name_base + '2c')
    self.activation = Activation("relu")
    self.add = Add(name= "")

  def call(self, inputs):
    X = inputs
    outputs = inputs

    outputs = self.conv2D_1(outputs)
    outputs = BatchNormalization(axis=3, name=self.bn_name_base + '2a')(outputs)
    outputs = self.activation(outputs)

    outputs = self.conv2D_2(outputs)
    outputs = BatchNormalization(axis=3, name=self.bn_name_base + '2b')(outputs)
    outputs = self.activation(outputs)

    outputs = self.conv2D_3(outputs)
    outputs = BatchNormalization(axis=3, name=self.bn_name_base + '2c')(outputs)

    outputs = self.add([outputs, X])

    outputs = self.activation(outputs)
    return outputs

class Conv_Block():

  def __init__(self, filters, kernel_size, stage, block, stride = 2):
    self.filters = filters
    self.kernel_size = kernel_size
    self.stride = stride
    self.conv_name_base = 'res' + str(stage) + block + '_branch'
    self.bn_name_base = 'bn' + str(stage) + block + '_branch'
    self.conv2D_1 = Conv2D(filters[0], kernel_size= (1, 1), strides=(stride,stride), padding= "valid", name=self.conv_name_base + '2a')
    self.conv2D_2 = Conv2D(filters[1], kernel_size= (kernel_size, kernel_size), strides=(1,1), padding= "same", name=self.conv_name_base + '2b')
    self.conv2D_3 = Conv2D(filters[2], kernel_size= (1, 1), strides=(1,1), padding= "valid", name=self.conv_name_base + '2c')
    self.conv2D_shortCut = Conv2D(filters[2], kernel_size= (1, 1), strides=(stride,stride), padding= "valid", name=self.conv_name_base + '1')
    self.activation = Activation("relu")
    self.add = Add()

  def call(self, inputs):
    X = inputs
    outputs = inputs

    outputs = self.conv2D_1(outputs)
    outputs = BatchNormalization(axis=3, name=self.bn_name_base + '2a')(outputs)
    outputs = self.activation(outputs)

    outputs = self.conv2D_2(outputs)
    outputs = BatchNormalization(axis=3, name=self.bn_name_base + '2b')(outputs)
    outputs = self.activation(outputs)

    outputs = self.conv2D_3(outputs)
    outputs = BatchNormalization(axis=3, name=self.bn_name_base + '2c')(outputs)

    X = self.conv2D_shortCut(X)
    X = BatchNormalization(axis=3, name=self.bn_name_base + '1')(X)

    outputs = self.add([outputs, X])

    outputs = self.activation(outputs)
    return outputs

class ResNet50():

  def __init__(self, includeTop = True, classes = 1000, input_shape= (224, 224, 3)):
    self.classes = classes
    self.includeTop = includeTop
    self.input_shape = input_shape

  def call(self, inputs):
    # outputs = keras.applications.resnet.preprocess_input(inputs)

    outputs = ZeroPadding2D((3, 3))(inputs)
    #stage 1
    outputs = Conv2D(64, kernel_size= (7, 7), strides=(2,2))(outputs)
    outputs = BatchNormalization(axis=3)(outputs)
    outputs = Activation("relu")(outputs)
    outputs = MaxPooling2D((3, 3), strides=(2, 2))(outputs)

    #stage 2
    outputs = Conv_Block([64, 64, 256], 3, stride= 1, stage = '2', block = 'a').call(outputs)
    outputs = I_Block([64, 64, 256], 3, stage = '2', block = 'b').call(outputs)
    outputs = I_Block([64, 64, 256], 3, stage = '2', block = 'c').call(outputs)

    #stage 3
    outputs = Conv_Block([128, 128, 512], 3, stage = '3', block = 'a').call(outputs)
    outputs = I_Block([128, 128, 512], 3, stage = '3', block = 'b').call(outputs)
    outputs = I_Block([128, 128, 512], 3, stage = '3', block = 'c').call(outputs)
    outputs = I_Block([128, 128, 512], 3, stage = '3', block = 'd').call(outputs)

    #stage 4
    outputs = Conv_Block([256, 256, 1024], 3, stage = '4', block = 'a').call(outputs)
    outputs = I_Block([256, 256, 1024], 3, stage = '4', block = 'b').call(outputs)
    outputs = I_Block([256, 256, 1024], 3, stage = '4', block = 'c').call(outputs)
    outputs = I_Block([256, 256, 1024], 3, stage = '4', block = 'd').call(outputs)
    outputs = I_Block([256, 256, 1024], 3, stage = '4', block = 'e').call(outputs)
    outputs = I_Block([256, 256, 1024], 3, stage = '4', block = 'f').call(outputs)

    #stage 5
    outputs = Conv_Block([512, 512, 2048], 3, stage = '5', block = 'a').call(outputs)
    outputs = I_Block([512, 512, 2048], 3, stage = '5', block = 'b').call(outputs)
    outputs = I_Block([512, 512, 2048], 3, stage = '5', block = 'c').call(outputs)

    #output layer
    if self.includeTop:
      outputs = AveragePooling2D((7, 7))(outputs)
      outputs = Flatten()(outputs)
      outputs =  Dense(self.classes, "softmax")(outputs)

    return outputs

  def getModel(self):
    inputs = Input(shape=self.input_shape)
    outputs = self.call(inputs)
    model = keras.Model(inputs= inputs, outputs= outputs, name= "ResNet50")

    return model


• Visual Geometry Group (VGG) :

In [0]:
class VGG16():
  def __init__(self, includeTop = True, classes = 1000, input_shape= (224, 224, 3)):
    self.input_shape = input_shape
    self.classes = classes
    self.includeTop = includeTop

  def call(self, inputs):
    # out = keras.applications.vgg16.preprocess_input(inputs)
    
    out = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_1')(inputs)
    out = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_1')(out)

    out = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(out)
    out = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_2')(out)

    out = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(out)
    out = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(out)
    out = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_3')(out)

    out = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(out)
    out = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(out)
    out = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_4')(out) 

    out = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(out)
    out = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(out)
    out = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_5')(out) 

    if self.includeTop:    
      out = Flatten(name='flatten')(out)
      out = Dense(4096, activation='relu', name='fully_connected_1')(out)
      out = Dense(4096, activation='relu', name='fully_connected_2')(out)
      out = Dense(self.classes, activation='softmax', name='predictions')(out)

    return out

  def getModel(self):
    inputs = Input(shape=self.input_shape)
    out = self.call(inputs)
    model = keras.Model(inputs= inputs, outputs= out, name= "VGG16")

    return model

Inception V3 :

In [0]:
class block_1():
  def __init__(self,
                     filters_1x1,
                     filters_3x3_reduce1,
                    filters_3x3_reduce2,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    self.filters_1x1 = filters_1x1
    self.filters_3x3_reduce1 = filters_3x3_reduce1
    self.filters_3x3_reduce2 = filters_3x3_reduce2
    self.filters_3x3 = filters_3x3
    self.filters_5x5_reduce = filters_5x5_reduce
    self.filters_5x5 = filters_5x5
    self.filters_pool_proj = filters_pool_proj
    self.name = name

  def call(self, inputs):
    x = inputs

    conv_1x1 = Conv2D(self.filters_1x1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    
    conv_3x3 = Conv2D(self.filters_3x3_reduce1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(self.filters_3x3_reduce2, (3, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3)
    conv_3x3 = Conv2D(self.filters_3x3, (3, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(self.filters_5x5_reduce, (1, 1), strides=(1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(self.filters_5x5, (5, 5), strides=(1, 1), padding='same', activation='relu')(conv_5x5)

    pool_proj = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(self.filters_pool_proj, (1, 1), strides=(1, 1), padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1,conv_5x5, conv_3x3,  pool_proj], axis=3, name=self.name)
    
    return output

class block_2():
  def __init__(self,
                     filters_1x1,
                     filters_3x3_reduce1,
                    filters_3x3_reduce2,
                     filters_3x3,
                     name=None):
    self.filters_1x1 = filters_1x1
    self.filters_3x3_reduce1 = filters_3x3_reduce1
    self.filters_3x3_reduce2 = filters_3x3_reduce2
    self.filters_3x3 = filters_3x3
    self.name = name

  def call(self, inputs):
    x = inputs

    conv_1x1 = Conv2D(self.filters_1x1, (3, 3),strides=(2, 2) , padding='valid', activation='relu')(x)
    
    conv_3x3 = Conv2D(self.filters_3x3_reduce1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(self.filters_3x3_reduce2, (3, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3)
    conv_3x3 = Conv2D(self.filters_3x3, (3, 3),strides=(2, 2) ,  padding='valid', activation='relu')(conv_3x3)

    pool_proj = MaxPooling2D((3, 3), strides=(2, 2))(x)

    output = concatenate([conv_1x1, conv_3x3, pool_proj], axis=3, name=self.name)
    
    return output

class block_3():
  def __init__(self,
                     filters_1x1,
                     filters_7x7_reduce1,
                     filters_7x7_reduce2,
                     filters_7x7,
                     filters_7x7dash_reduce1,
                     filters_7x7dash_reduce2,
                     filters_7x7dash_reduce3,
                    filters_7x7dash_reduce4,
                     filters_7x7dash,
                     filters_pool_proj,
                     name=None):
    self.filters_1x1 = filters_1x1
    self.filters_7x7_reduce1 = filters_7x7_reduce1
    self.filters_7x7_reduce2 = filters_7x7_reduce2
    self.filters_7x7 = filters_7x7
    self.filters_7x7dash_reduce1 = filters_7x7dash_reduce1
    self.filters_7x7dash_reduce2 = filters_7x7dash_reduce2
    self.filters_7x7dash_reduce3 = filters_7x7dash_reduce3
    self.filters_7x7dash_reduce4 = filters_7x7dash_reduce4
    self.filters_7x7dash = filters_7x7dash
    self.filters_pool_proj = filters_pool_proj
    self.name = name

  def call(self, inputs):
    x = inputs

    conv_1x1 = Conv2D(self.filters_1x1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    
    conv_7x7 = Conv2D(self.filters_7x7_reduce1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    conv_7x7 = Conv2D(self.filters_7x7_reduce2, (1, 7),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7)
    conv_7x7 = Conv2D(self.filters_7x7, (7, 1),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7)

    conv_7x7dash = Conv2D(self.filters_7x7dash_reduce1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    conv_7x7dash = Conv2D(self.filters_7x7dash_reduce2, (7, 1),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7dash)
    conv_7x7dash = Conv2D(self.filters_7x7dash_reduce3, (1, 7),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7dash)
    conv_7x7dash = Conv2D(self.filters_7x7dash_reduce4, (7, 1),strides=(1, 1) , padding='same', activation='relu')(conv_7x7dash)
    conv_7x7dash = Conv2D(self.filters_7x7dash, (1, 7),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7dash)

    pool_proj = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(self.filters_pool_proj, (1, 1),strides=(1, 1) ,  padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1, conv_7x7, conv_7x7dash, pool_proj], axis=3, name=self.name)
    
    return output

class block_4():
  def __init__(self,
                    filters_3x3_reduce1,               
                     filters_3x3,
                     filters_7x7_reduce1,
                    filters_7x7_reduce2,
                      filters_7x7_reduce3,
                     filters_7x7,
                     name=None):
    self.filters_3x3 = filters_3x3
    self.filters_3x3_reduce1 = filters_3x3_reduce1
    self.filters_7x7_reduce1 = filters_7x7_reduce1
    self.filters_7x7_reduce2 = filters_7x7_reduce2
    self.filters_7x7_reduce3 = filters_7x7_reduce3
    self.filters_7x7 = filters_7x7
    self.name = name

  def call(self, inputs):
    x = inputs

    conv_3x3 = Conv2D(self.filters_3x3_reduce1, (1, 1),strides=(1, 1) ,  padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(self.filters_3x3, (3, 3),strides=(2, 2) ,  padding='valid', activation='relu')(conv_3x3)

    conv_7x7 = Conv2D(self.filters_7x7_reduce1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)
    conv_7x7 = Conv2D(self.filters_7x7_reduce2, (1, 7),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7)
    conv_7x7 = Conv2D(self.filters_7x7_reduce3, (7, 1),strides=(1, 1) ,  padding='same', activation='relu')(conv_7x7)
    conv_7x7 = Conv2D(self.filters_7x7, (3, 3),strides=(2, 2) ,  padding='valid', activation='relu')(conv_7x7)

    pool_proj = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    output = concatenate([conv_3x3, conv_7x7, pool_proj], axis=3, name=self.name)
    
    return output

class block_5():
  def __init__(self,
               filters_1x1,
                    filters_3x3_reduce1,               
                    filters_3x3_reduce2, 
               filters_3x3dash_reduce1,
               filters_3x3dash_reduce2,
               filters_3x3dash_reduce3,
               filters_3x3dash,              
                     filters_3x3,
                     filters_pool_proj,
                     name=None):
    self.filters_1x1 = filters_1x1
    self.filters_3x3 = filters_3x3
    self.filters_3x3_reduce1 = filters_3x3_reduce1
    self.filters_3x3_reduce2 = filters_3x3_reduce2
    self.filters_3x3dash = filters_3x3dash
    self.filters_3x3dash_reduce1 = filters_3x3dash_reduce1
    self.filters_3x3dash_reduce2 = filters_3x3dash_reduce2
    self.filters_3x3dash_reduce3 = filters_3x3dash_reduce3
    self.filters_pool_proj = filters_pool_proj
    self.name = name

  def call(self, inputs):
    x = inputs
    conv_1x1 = Conv2D(self.filters_1x1, (1, 1),strides=(1, 1) , padding='same', activation='relu')(x)

    conv_3x3 = Conv2D(self.filters_3x3_reduce1, (1, 1),strides=(1, 1) ,  padding='same', activation='relu')(x)
    conv_3x3_1 = Conv2D(self.filters_3x3_reduce2, (1, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3)
    conv_3x3_2 = Conv2D(self.filters_3x3, (1, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3)
    conv_3x3 = concatenate([conv_3x3_1, conv_3x3_2], axis=3, name=self.name+'_1')


    conv_3x3dash = Conv2D(self.filters_3x3dash_reduce1, (1, 1),strides=(1, 1) ,  padding='same', activation='relu')(x)
    conv_3x3dash = Conv2D(self.filters_3x3dash_reduce2, (3, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3dash)
    conv_3x3dash_1 = Conv2D(self.filters_3x3dash_reduce3, (1, 3),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3dash)
    conv_3x3dash_2 = Conv2D(self.filters_3x3dash, (3, 1),strides=(1, 1) ,  padding='same', activation='relu')(conv_3x3dash)
    conv_3x3dash = concatenate([conv_3x3dash_1, conv_3x3dash_2], axis=3, name=self.name+'_2')

    pool_proj = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(self.filters_pool_proj, (1, 1),strides=(1, 1) ,  padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_3x3dash, pool_proj], axis=3, name=self.name)
    
    return output

class InceptionV3():
  def __init__(self, includeTop = True, classes = 1000, input_shape= (299, 299, 3)):
    self.input_shape = input_shape
    self.classes = classes
    self.includeTop = includeTop

  def call(self, inputs):
    # x = keras.applications.inception_v3.preprocess_input(inputs)
    
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='valid', name='conv1_1')(inputs)
    x = Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='valid', name='conv1_2')(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_3')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pooling_1')(x)

    x = Conv2D(80, (1, 1), strides=(1, 1), activation='relu', padding='valid', name='conv2_1')(x)
    x = Conv2D(192, (3, 3), strides=(1, 1), activation='relu', padding='valid', name='conv2_2')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pooling_2')(x)

    x = block_1(filters_1x1=64,
                     filters_3x3_reduce1=64,
                     filters_3x3_reduce2=96,
                     filters_3x3=96,
                     filters_5x5_reduce=48,
                     filters_5x5=64,
                     filters_pool_proj=32,
                     name='inception_3a').call(x)

    x = block_1(filters_1x1=64,
                     filters_3x3_reduce1=64,
                     filters_3x3_reduce2=96,
                     filters_3x3=96,
                     filters_5x5_reduce=48,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_3b').call(x)
    x = block_1(filters_1x1=64,
                     filters_3x3_reduce1=64,
                     filters_3x3_reduce2=96,
                     filters_3x3=96,
                     filters_5x5_reduce=48,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_3c').call(x)

    x = block_2(filters_1x1=384,
                     filters_3x3_reduce1=64,
                     filters_3x3_reduce2=96,
                     filters_3x3=96,
                     name='inception_4a').call(x)
    
    x = block_3(filters_1x1=192,
                     filters_7x7_reduce1=128,
                     filters_7x7_reduce2=128,
                     filters_7x7=192,
                     filters_7x7dash_reduce1=128,
                     filters_7x7dash_reduce2=128,
                     filters_7x7dash_reduce3=128,
                    filters_7x7dash_reduce4=128,
                     filters_7x7dash=192,
                     filters_pool_proj=192,
                     name='inception_5a').call(x)
    x = block_3(filters_1x1=192,
                     filters_7x7_reduce1=160,
                     filters_7x7_reduce2=160,
                     filters_7x7=192,
                     filters_7x7dash_reduce1=160,
                     filters_7x7dash_reduce2=160,
                     filters_7x7dash_reduce3=160,
                    filters_7x7dash_reduce4=160,
                     filters_7x7dash=192,
                     filters_pool_proj=192,
                     name='inception_5b').call(x)
    x = block_3(filters_1x1=192,
                     filters_7x7_reduce1=160,
                     filters_7x7_reduce2=160,
                     filters_7x7=192,
                     filters_7x7dash_reduce1=160,
                     filters_7x7dash_reduce2=160,
                     filters_7x7dash_reduce3=160,
                    filters_7x7dash_reduce4=160,
                     filters_7x7dash=192,
                     filters_pool_proj=192,
                     name='inception_5c').call(x)
    x = block_3( filters_1x1=192,
                     filters_7x7_reduce1=192,
                     filters_7x7_reduce2=192,
                     filters_7x7=192,
                     filters_7x7dash_reduce1=192,
                     filters_7x7dash_reduce2=192,
                     filters_7x7dash_reduce3=192,
                    filters_7x7dash_reduce4=192,
                     filters_7x7dash=192,
                     filters_pool_proj=192,
                     name='inception_5d').call(x)

    x = block_4(filters_3x3_reduce1=192,               
                filters_3x3=320,
                filters_7x7_reduce1=192,
                filters_7x7_reduce2=192,
                filters_7x7_reduce3=192,
                filters_7x7=192,
                name='inception_6a').call(x)

    x = block_5(filters_1x1=320,
              filters_3x3_reduce1=384,               
              filters_3x3_reduce2=384, 
               filters_3x3dash_reduce1=448,
               filters_3x3dash_reduce2=384,
               filters_3x3dash_reduce3=384,
               filters_3x3dash=384,              
                filters_3x3=384,
                filters_pool_proj=192,
                name='inception_7a').call(x)

    x = block_5(filters_1x1=320,
              filters_3x3_reduce1=384,               
              filters_3x3_reduce2=384, 
               filters_3x3dash_reduce1=448,
               filters_3x3dash_reduce2=384,
               filters_3x3dash_reduce3=384,
               filters_3x3dash=384,              
                filters_3x3=384,
                filters_pool_proj=192,
                name='inception_7b').call(x)

    if self.includeTop:
      x = GlobalAveragePooling2D(name='avg_pool')(x)
      x = Dense(self.classes, activation='softmax', name='predictions')(x)

    return x

  def getModel(self):
    inputs = Input(shape=self.input_shape)
    out = self.call(inputs)
    model = keras.Model(inputs= inputs, outputs= out, name= "InceptionV3")

    return model

In [0]:
class MRNet():
  def __init__(self, CNNModel, name = ""):
    self.CNNModel = CNNModel
    self.input_shape = CNNModel.input_shape[1:]
    self.name = name

  def call(self, inputs):
    outputs = self.CNNModel(inputs)
    outputs = GlobalAveragePooling2D()(outputs)
    # max pooling
    outputs = tf.math.reduce_max(outputs, axis= 0, keepdims= True)
    # dense
    outputs = Dense(512, kernel_regularizer=keras.regularizers.l2(0.0001))(outputs)
    outputs = Dropout(0.5)(outputs)
    
    outputs = Dense(1, "sigmoid")(outputs)
    return outputs

  def getModel(self):

    inputs = Input(shape=self.input_shape)
    outputs = self.call(inputs)

    model = keras.Model(inputs= inputs, outputs= outputs, name= "MRNet_" + self.CNNModel.name + "_" + self.name)
    return model

In [0]:
# modelCNN = VGG16(includeTop=False, input_shape= (224, 224, 3)).getModel()
# modelCNN.summary()

Model: "VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pooling_1 (MaxPooling2D)     (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pooling_2 (MaxPooling2D)     (None, 56, 56, 128)       0     

In [0]:
# modelCNN = ResNet50(includeTop=False, input_shape= (224, 224, 3)).getModel()
# modelCNN.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
___________________________________________________________________________________________

In [0]:
# MRnetModel = MRNet(modelCNN).getModel()
# MRnetModel.summary()

Model: "MRNet_ResNet50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
ResNet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
tf_op_layer_Max (TensorFlowO [(1, 2048)]               0         
_________________________________________________________________
dense (Dense)                (1, 512)                  1049088   
_________________________________________________________________
dropout (Dropout)            (1, 512)                  0         
_________________________________________________________________
dense_1 (Dense)              (1, 1)                 

In [0]:
# modelVGG = VGG16(includeTop=False, input_shape= (224, 224, 3)).getModel()
# modelVGG = MRNet(modelVGG).getModel()
# modelVGG.summary()

Model: "MRNet_VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
VGG16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
tf_op_layer_Max_1 (TensorFlo [(1, 512)]                0         
_________________________________________________________________
dense_2 (Dense)              (1, 512)                  262656    
_________________________________________________________________
dropout_1 (Dropout)          (1, 512)                  0         
_________________________________________________________________
dense_3 (Dense)              (1, 1)                    

In [0]:
#modelInceptionV3 = InceptionV3(includeTop=True, input_shape= (299, 299, 3)).getModel()
# modelInceptionV3 = MRNet(modelInceptionV3).getModel()
#modelInceptionV3.summary()

Model: "InceptionV3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 147, 147, 32) 9248        conv1_1[0][0]                    
__________________________________________________________________________________________________
conv1_3 (Conv2D)                (None, 147, 147, 64) 18496       conv1_2[0][0]                    
________________________________________________________________________________________

In [0]:
class TripleMRNet():
  def __init__(self, MRNet_sagittal, MRNet_coronal, MRNet_axial, logisticRegression, name):
    self.MRNet_sagittal = MRNet_sagittal
    self.MRNet_coronal = MRNet_coronal
    self.MRNet_axial = MRNet_axial
    self.logisticRegression = logisticRegression
    self.input_shape = MRNet_sagittal.input_shape[1:]
    self.name = name

  def call(self, inputs):
    sagittal_output = self.MRNet_sagittal(inputs[0])
    coronal_output = self.MRNet_coronal(inputs[1])
    axial_output = self.MRNet_axial(inputs[2])

    outputs = concatenate([sagittal_output,coronal_output, axial_output], axis=1)
    outputs = logisticRegression(outputs)

    return outputs


  def getModel(self):
    sagittal_input = Input(shape=self.input_shape)
    coronal_input = Input(shape=self.input_shape)
    axial_input = Input(shape=self.input_shape)

    inputs = [sagittal_input, coronal_input, axial_input]

    outputs = self.call(inputs)

    model = keras.Model(inputs= inputs, outputs= outputs, name= "MRNet_" + self.name)

    return model

In [0]:
class LogisticRegression():
  def __init__(self, name= "LogisticRegression", input_shape = (3, )):
    self.name = name
    self.input_shape = input_shape

  def call(self, inputs):
    return Dense(1, activation='sigmoid')(inputs)

  def getModel(self):
    inputs = Input(shape= self.input_shape)
    outputs = self.call(inputs)
    model = keras.Model(inputs= inputs, outputs= outputs, name= self.name)

    return model

In [0]:
# MRNet_sagittal = MRNet(ResNet50(includeTop=False, input_shape= (224, 224, 3)).getModel(), name= "sagittal").getModel()
# MRNet_coronal = MRNet(ResNet50(includeTop=False, input_shape= (224, 224, 3)).getModel(), name= "coronal").getModel()
# MRNet_axial = MRNet(ResNet50(includeTop=False, input_shape= (224, 224, 3)).getModel(), name= "axial").getModel()
# logisticRegression = LogisticRegression().getModel()

In [0]:
# MRNet_ACL = TripleMRNet(MRNet_sagittal, MRNet_coronal, MRNet_axial, logisticRegression, name= "ACL").getModel()

In [0]:
# MRNet_ACL.summary()

Model: "MRNet_ACL"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
MRNet_ResNet50_sagittal (Model) (1, 1)               24637313    input_12[0][0]                   
__________________________________________________________________________________________